In [1]:
scm_df = spark.read.table("silver_scm")

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 3, Finished, Available, Finished)

In [2]:
from pyspark.sql.types import *
from delta.tables import *

DeltaTable.createIfNotExists(spark) \
          .tableName("gold_scm_date") \
          .addColumn("order_date", DateType()) \
          .addColumn("day", IntegerType()) \
          .addColumn("month", IntegerType()) \
          .addColumn("year", IntegerType()) \
          .addColumn("yyyymm", StringType()) \
          .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 4, Finished, Available, Finished)

In [ ]:
from pyspark.sql.functions import col, dayofmonth, month, year, date_format

dfdimDate_gold = scm_df.dropDuplicates(["order_date"]) \
                    .select(col("order_date"), \
                        dayofmonth("order_date").alias("day"), \
                        month("order_date").alias("month"), \
                        year("order_date").alias("year"), \
                        date_format(col("order_date"), "yyyyMM").alias("yyyymm"), \
                    ) \
                    .orderBy("order_date")
display(dfdimDate_gold.head(3))

In [4]:
deltaTable = DeltaTable.forPath(spark, 'Tables/gold_scm_date')
    
dfUpdates = dfdimDate_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.order_date = updates.order_date'
  ) \
  .whenMatchedUpdate(set =
    {
          
    }
  ) \
  .whenNotMatchedInsert(values =
    {
      "order_date": "updates.order_date",
      "day": "updates.day",
      "month": "updates.month",
      "year": "updates.year",
      "yyyymm": "updates.yyyymm"
    }
  ) \
  .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 6, Finished, Available, Finished)

In [5]:
scm_df.columns

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 7, Finished, Available, Finished)

['type',
 'shipping_days_actual',
 'shipping_days_scheduled',
 'sales_actual',
 'delivery_status',
 'late_delivery_risk',
 'category_id',
 'category_name',
 'customer_city',
 'customer_id',
 'customer_segment',
 'customer_state',
 'department_name',
 'latitude',
 'longitude',
 'market',
 'order_city',
 'order_country',
 'order_date',
 'order_id',
 'order_dis',
 'order_dis_rate',
 'order_price',
 'order_qty',
 'order_item_total',
 'order_profit',
 'order_region',
 'order_state',
 'order_status',
 'product_id',
 'product_name',
 'product_price',
 'shipping_date',
 'shipping_mode',
 'created_ts',
 'modified_ts',
 'customer_fullname']

In [6]:
DeltaTable.createIfNotExists(spark) \
          .tableName("gold_scm_customer") \
          .addColumn("customer_id", StringType()) \
          .addColumn("customer_segment", StringType()) \
          .addColumn("customer_fullname", StringType()) \
          .addColumn("customer_city", StringType()) \
          .addColumn("customer_state", StringType()) \
          .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 8, Finished, Available, Finished)

In [7]:
dim_customer_df = spark.read.table("gold_scm_customer")
new_customer_df = scm_df.dropDuplicates(["customer_id"])
dfdimCustomer_gold = new_customer_df.join(
    dim_customer_df,
    on = "customer_id",
    how = "left_anti"
)
display(dfdimCustomer_gold.head(3))

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2cd32f4c-56d9-4655-85f1-0b893b2f4a23)

In [8]:
deltaTable = DeltaTable.forPath(spark, 'Tables/gold_scm_customer')
    
dfUpdates = dfdimCustomer_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.customer_id = updates.customer_id'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "customer_id": "updates.customer_id",
      "customer_segment": "updates.customer_segment",
      "customer_fullname": "updates.customer_fullname",
      "customer_city": "updates.customer_city",
      "customer_state": "updates.customer_state"
    }
  ) \
  .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 10, Finished, Available, Finished)

In [9]:
DeltaTable.createIfNotExists(spark) \
          .tableName("gold_scm_product") \
          .addColumn("catetory_id", StringType()) \
          .addColumn("category_name", StringType()) \
          .addColumn("department_name", StringType()) \
          .addColumn("product_id", StringType()) \
          .addColumn("product_name", LongType()) \
          .addColumn("product_price", StringType()) \
          .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 11, Finished, Available, Finished)

In [10]:
from pyspark.sql.functions import col

dim_product_df = spark.read.table("gold_scm_product")
new_product_df = scm_df.dropDuplicates(["product_id"])
dfdimProduct_gold = new_product_df.join(
    dim_product_df,
    on = "product_id",
    how = "left_anti"
)
display(dfdimProduct_gold.head(3))

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 78cf5d3e-fd91-412e-b004-641ebbc98475)

In [11]:
deltaTable = DeltaTable.forPath(spark, 'Tables/gold_scm_product')
            
dfUpdates = dfdimProduct_gold
            
deltaTable.alias('gold') \
  .merge(
        dfUpdates.alias('updates'),
        'gold.product_id = updates.product_id'
        ) \
        .whenMatchedUpdate(set =
        {
               
        }
        ) \
        .whenNotMatchedInsert(values =
         {
          "catetory_id": "updates.category_id",
          "category_name": "updates.category_name",
          "department_name": "updates.department_name",
          "product_id":"updates.product_id",
          "product_name": "updates.product_name",
          "product_price": "updates.product_price"
          }
          ) \
          .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 13, Finished, Available, Finished)

In [12]:
DeltaTable.createIfNotExists(spark) \
    .tableName("gold_scm_location") \
    .addColumn("location_id", StringType()) \
    .addColumn("order_city", StringType()) \
    .addColumn("order_state", StringType()) \
    .addColumn("order_region", StringType()) \
    .addColumn("order_country", StringType()) \
    .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 14, Finished, Available, Finished)

In [13]:
from pyspark.sql.functions import monotonically_increasing_id, coalesce, max, lit, col

dim_location_df = spark.read.table("gold_scm_location")
new_location_df = scm_df.select("order_city", "order_state", "order_region", "order_country").dropDuplicates()
dfdimLocation_gold = new_location_df.alias("new").join(
    dim_location_df.alias("dim"),
    (col("new.order_state") == col("dim.order_state")) &
    (col("new.order_state") == col("dim.order_state")) &
    (col("new.order_region") == col("dim.order_region")) &
    (col("new.order_country") == col("dim.order_country")),
    how="left_anti"
)

max_id = dim_location_df.select(coalesce(max("location_id"), lit(0)).alias("max_id")).first()["max_id"]

dfdimLocation_gold = dfdimLocation_gold.withColumn("location_id", monotonically_increasing_id() + max_id + 1)

dfdimLocation_gold = dfdimLocation_gold.select("location_id", "order_city", "order_state", "order_region", "order_country")

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 15, Finished, Available, Finished)

In [14]:
deltaTable = DeltaTable.forPath(spark, 'Tables/gold_scm_location')
            
dfUpdates = dfdimLocation_gold
            
deltaTable.alias('gold') \
  .merge(
        dfUpdates.alias('updates'),
        'gold.location_id = updates.location_id'
        ) \
        .whenMatchedUpdate(set =
        {
               
        }
        ) \
        .whenNotMatchedInsert(values =
         {
          "location_id": "updates.location_id",
          "order_city": "updates.order_city",
          "order_state": "updates.order_state",
          "order_region":"updates.order_region",
          "order_country": "updates.order_country"
          }
          ) \
          .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 16, Finished, Available, Finished)

In [15]:
DeltaTable.createIfNotExists(spark) \
    .tableName("gold_scm_txn") \
    .addColumn("customer_id", StringType()) \
    .addColumn("product_id", StringType()) \
    .addColumn("category_id", StringType()) \
    .addColumn("location_id", StringType()) \
    .addColumn("order_id", StringType()) \
    .addColumn("order_date", DateType()) \
    .addColumn("shipping_date", DateType()) \
    .addColumn("shipping_mode", StringType()) \
    .addColumn("shipping_days_actual", IntegerType()) \
    .addColumn("shipping_days_scheduled", IntegerType()) \
    .addColumn("sales_actual", FloatType()) \
    .addColumn("order_profit", FloatType()) \
    .addColumn("order_item_total", FloatType()) \
    .addColumn("order_price", FloatType()) \
    .addColumn("order_dis", FloatType()) \
    .addColumn("order_dis_rate", FloatType()) \
    .addColumn("order_qty", IntegerType()) \
    .addColumn("order_status", StringType()) \
    .addColumn("delivery_status", StringType()) \
    .addColumn("late_delivery_risk", StringType()) \
    .addColumn("market", StringType()) \
    .execute()

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 17, Finished, Available, Finished)

In [16]:
modi_scm_df = scm_df.join(
    dfdimLocation_gold,
    on = ["order_city", "order_state", "order_region", "order_country"],
    how = "left"
)
modi_scm_df = modi_scm_df.dropDuplicates(["customer_id", "product_id", "category_id", "order_id", "location_id"])
display(modi_scm_df.head(3))

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d114a6e9-1e7a-4439-9910-52a9e2053b24)

In [17]:
modi_scm_df.write \
    .option("header", "true") \
    .mode("overwrite") \
    .csv("Files/bronze_scm/")

StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 19, Finished, Available, Finished)

In [18]:
from pyspark.sql.functions import col

deltaTable = DeltaTable.forPath(spark, "Tables/gold_scm_txn")

dfUpdates = modi_scm_df

deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.customer_id = updates.customer_id and gold.product_id = updates.product_id and gold.category_id = updates.category_id and gold.order_id = updates.order_id'
        ) \
  .whenMatchedUpdate(set={
    "shipping_date": col("updates.shipping_date"),
    "order_status": col("updates.order_status"),
     }) \
  .whenNotMatchedInsert(values={
    "customer_id": col("updates.customer_id"),
    "product_id": col("updates.product_id"),
    "category_id": col("updates.category_id"),
    "location_id": col("updates.location_id"),
    "order_id": col("updates.order_id"),
    "order_date": col("updates.order_date"),
    "shipping_date": col("updates.shipping_date"),
    "shipping_mode": col("updates.shipping_mode"),
    "shipping_days_actual": col("updates.shipping_days_actual"),
    "shipping_days_scheduled": col("updates.shipping_days_scheduled"),
    "sales_actual": col("updates.sales_actual"),
    "order_profit": col("updates.order_profit"),
    "order_item_total": col("updates.order_item_total"),
    "order_price": col("updates.order_price"),
    "order_dis": col("updates.order_dis"),
    "order_dis_rate": col("updates.order_dis_rate"),
    "order_qty": col("updates.order_qty"),
    "order_status": col("updates.order_status"),
    "delivery_status": col("updates.delivery_status"),
    "late_delivery_risk": col("updates.late_delivery_risk"),
    "market": col("updates.market")
  }) \
  .execute()


StatementMeta(, dea0e8df-12a8-4288-8656-a0bcadffad97, 20, Finished, Available, Finished)